[profiles](data/profiles.csv)

LinkedDictionary & LinkedQueue from template

In [ ]:
class LinkedDictionary:
    # Simple dictionary implementation
    def __init__(self):
        self.dict = {}

    def add(self, key, value):
        self.dict[key] = value

    def remove(self, key):
        if key in self.dict:
            del self.dict[key]

    def get_value(self, key):
        return self.dict.get(key, None)

    def get_keys(self):
        return self.dict.keys()

class LinkedQueue:
    # Simple queue implementation using list
    def __init__(self):
        self.queue = []

    def enqueue(self, item):
        self.queue.append(item)

    def dequeue(self):
        if not self.is_empty():
            return self.queue.pop(0)
        return None

    def is_empty(self):
        return len(self.queue) == 0

ProfileManager: FE

In [ ]:
import csv
from graphviz import Digraph

class ProfileManager:
    def __init__(self):
        self.profiles = LinkedDictionary()
        self.connections = {}  # Dictionary to store connections between profiles

    def add_profile(self, name, location, relationship_status, age, occupation, astrological_sign, status=""):
        if name in self.profiles.get_keys():
            print(f"Profile for {name} already exists.")
            return
        self.profiles.add(name, {
            "location": location,
            "relationship_status": relationship_status,
            "age": age,
            "occupation": occupation,
            "astrological_sign": astrological_sign,
            "status": status,
            "friends": []
        })
        self.connections[name] = []

    def get_profile(self, name):
        return self.profiles.get_value(name)

    def remove_profile(self, name):
        self.profiles.remove(name)
        if name in self.connections:
            del self.connections[name]
        # Remove the profile from all other profiles' friend lists
        for friends in self.connections.values():
            if name in friends:
                friends.remove(name)

    def connect_profiles(self, name1, name2, weight=0):
        if name1 not in self.connections or name2 not in self.connections:
            print(f"Cannot connect profiles: {name1} or {name2} does not exist.")
            return
        if name2 not in self.connections[name1]:
            self.connections[name1].append(name2)
        if name1 not in self.connections[name2]:
            self.connections[name2].append(name1)

    def display_profiles(self):
        for name in self.profiles.get_keys():
            print(name)

    def display_profile_details(self, name):
        profile = self.get_profile(name)
        if profile:
            print(f"Name: {name}")
            for key, value in profile.items():
                print(f"{key.capitalize()}: {value}")
        else:
            print(f"Profile for {name} not found.")

    def get_friends_of_friends(self, name):
        if name not in self.connections:
            print(f"Profile for {name} does not exist.")
            return []
        friends = self.connections[name]
        friends_of_friends = set()
        for friend in friends:
            friends_of_friends.update(self.connections.get(friend, []))
        friends_of_friends.discard(name)  # Remove the original user if present
        return list(friends_of_friends)

    def read_profiles_from_csv(self, file_path):
        try:
            with open(file_path, mode='r') as csv_file:
                reader = csv.DictReader(csv_file)
                next(reader)
                for row in reader:
                    name = row["name"]
                    friends = row["friends"].split('|') if row["friends"] else []
                    self.add_profile(
                        name=name,
                        location=row["location"],
                        relationship_status=row["relationship_status"],
                        age=int(row["age"]),
                        occupation=row["occupation"],
                        astrological_sign=row["astrological_sign"],
                        status=row["status"]
                    )
                    for friend in friends:
                        self.connect_profiles(name, friend)
        except Exception as e:
            print(f"Error reading CSV file: {e}")

    def create_user_graph(self, current_user, depth=1):
        if current_user not in self.connections:
            print(f"Profile for {current_user} does not exist.")
            return
        graph = Digraph()
        visited = set()

        def dfs(user, current_depth):
            if user in visited or current_depth > depth:
                return
            visited.add(user)
            graph.node(user)
            for friend in self.connections[user]:
                graph.node(friend)
                graph.edge(user, friend)
                dfs(friend, current_depth + 1)

        dfs(current_user, 0)
        graph.render(f"{current_user}_network", format="png", cleanup=True)
        print(f"Graph for {current_user}'s network created.")

# Example usage
# pm = ProfileManager()
# pm.read_profiles_from_csv('profiles.csv')
# pm.create_user_graph('Alice', depth=2)


User Profile: FE

In [ ]:
class UserProfile:
    def __init__(self, name, location, relationship_status, age, occupation, astrological_sign, status=""):
        self.name = name
        self.location = location
        self.relationship_status = relationship_status
        self.age = age
        self.occupation = occupation
        self.astrological_sign = astrological_sign
        self.status = status
        self.friends = []

    def get_name(self):
        return self.name

    def get_location(self):
        return self.location

    def get_relationship_status(self):
        return self.relationship_status

    def get_age(self):
        return self.age

    def get_occupation(self):
        return self.occupation

    def get_astrological_sign(self):
        return self.astrological_sign

    def get_status(self):
        return self.status

    def set_status(self, status):
        self.status = status

    def get_friends(self):
        return self.friends

    def add_friend(self, friend_profile):
        if friend_profile not in self.friends:
            self.friends.append(friend_profile)
        else:
            print(f"{friend_profile.get_name()} is already a friend.")

    def remove_friend(self, friend_profile):
        if friend_profile in self.friends:
            self.friends.remove(friend_profile)
        else:
            print(f"{friend_profile.get_name()} is not a friend.")

    def print_details(self):
        print(f"Name: {self.name}")
        print(f"Location: {self.location}")
        print(f"Relationship Status: {self.relationship_status}")
        print(f"Age: {self.age}")
        print(f"Occupation: {self.occupation}")
        print(f"Astrological Sign: {self.astrological_sign}")
        print(f"Status: {self.status}")
        print(f"Friends: {[friend.get_name() for friend in self.friends]}")

# Example usage
# user1 = UserProfile("Alice", "Seattle", "Single", 30, "Engineer", "Aries", "Feeling happy")
# user2 = UserProfile("Bob", "San Francisco", "Married", 35, "Artist", "Taurus", "Excited for the weekend")
# user1.add_friend(user2)
# user1.print_details()


UndirectedGraph: FE

In [ ]:
from collections import deque

class UndirectedGraph:
    def __init__(self):
        self.adjacency_list = {}

    def add_vertex(self, key):
        if key not in self.adjacency_list:
            self.adjacency_list[key] = {}

    def get_vertex(self, key):
        return self.adjacency_list.get(key)

    def add_edge(self, from_key, to_key, weight=0):
        if from_key not in self.adjacency_list:
            self.add_vertex(from_key)
        if to_key not in self.adjacency_list:
            self.add_vertex(to_key)
        self.adjacency_list[from_key][to_key] = weight
        self.adjacency_list[to_key][from_key] = weight  # Because it's undirected

    def get_vertices(self):
        return list(self.adjacency_list.keys())

    def contains(self, key):
        return key in self.adjacency_list

    def clear(self):
        self.adjacency_list.clear()

    def is_empty(self):
        return len(self.adjacency_list) == 0

    def size(self):
        return len(self.adjacency_list)

    def get_edges(self):
        edges = []
        visited = set()
        for vertex, neighbors in self.adjacency_list.items():
            for neighbor, weight in neighbors.items():
                if (neighbor, vertex) not in visited:  # Avoid duplicates
                    edges.append((vertex, neighbor, weight))
                    visited.add((vertex, neighbor))
        return edges

    def bfs(self, start):
        if start not in self.adjacency_list:
            print(f"Vertex {start} does not exist.")
            return []
        visited = set()
        queue = deque([start])
        result = []

        while queue:
            current = queue.popleft()
            if current not in visited:
                visited.add(current)
                result.append(current)
                queue.extend(self.adjacency_list[current].keys() - visited)

        return result

    def dfs(self, start):
        if start not in self.adjacency_list:
            print(f"Vertex {start} does not exist.")
            return []
        visited = set()
        stack = [start]
        result = []

        while stack:
            current = stack.pop()
            if current not in visited:
                visited.add(current)
                result.append(current)
                stack.extend(self.adjacency_list[current].keys() - visited)

        return result

# Example usage
# graph = UndirectedGraph()
# graph.add_vertex("Alice")
# graph.add_vertex("Bob")
# graph.add_edge("Alice", "Bob", 5)
# graph.add_edge("Alice", "Charlie")
# graph.add_edge("Bob", "Dana")
# print("Vertices:", graph.get_vertices())
# print("Edges:", graph.get_edges())
# print("BFS from Alice:", graph.bfs("Alice"))
# print("DFS from Alice:", graph.dfs("Alice"))


Vertex: FE

In [ ]:
class Vertex:
    def __init__(self, key):
        self.id = key
        self.neighbors = {}  # Dictionary of neighbors and their edge weights

    def add_neighbor(self, nbr, weight=0):
        """Adds a neighbor with an optional weight."""
        self.neighbors[nbr] = weight

    def get_connections(self):
        """Returns a list of neighbors."""
        return self.neighbors.keys()

    def get_id(self):
        """Returns the vertex ID."""
        return self.id

    def get_weight(self, nbr):
        """Gets the weight of the edge to a neighbor."""
        return self.neighbors.get(nbr, None)

# Example usage
# v1 = Vertex("Alice")
# v2 = Vertex("Bob")
# v1.add_neighbor(v2, 5)
# print(v1.get_id())  # Output: Alice
# print([nbr.get_id() for nbr in v1.get_connections()])  # Output: ['Bob']
# print(v1.get_weight(v2))  # Output: 5


SocialMediaApp: FE

In [ ]:
import csv
from graphviz import Graph

class SocialMediaApp:
    def __init__(self):
        self.profile_manager = ProfileManager()
        self.current_user = None

    def create_profile(self):
        """Prompt user to create a new profile."""
        print("\n--- Create a Profile ---")
        name = input("Enter name: ")
        location = input("Enter location: ")
        relationship_status = input("Enter relationship status: ")
        age = int(input("Enter age: "))
        occupation = input("Enter occupation: ")
        astrological_sign = input("Enter astrological sign: ")
        status = input("Enter a status message (optional): ")

        self.profile_manager.add_profile(
            name=name, location=location, relationship_status=relationship_status,
            age=age, occupation=occupation, astrological_sign=astrological_sign, status=status
        )
        print(f"Profile for {name} created successfully!")

        # Set the current user if it's the first profile
        if not self.current_user:
            self.current_user = self.profile_manager.get_profile(name)
            print(f"Current user set to {name}.")

    def modify_profile(self):
        """Modify the current user's profile."""
        if not self.current_user:
            print("No current user selected.")
            return
        print("\n--- Modify Profile ---")
        self.current_user.print_details()

        print("What would you like to modify?")
        print("1. Status")
        print("2. Location")
        print("3. Relationship status")
        choice = input("Enter your choice: ")

        if choice == "1":
            status = input("Enter a new status: ")
            self.current_user.set_status(status)
        elif choice == "2":
            location = input("Enter a new location: ")
            self.current_user.location = location
        elif choice == "3":
            relationship_status = input("Enter a new relationship status: ")
            self.current_user.relationship_status = relationship_status
        else:
            print("Invalid choice.")
            return

        print("Profile updated successfully.")

    def view_all_profiles(self):
        """View all profiles in BFS or DFS order."""
        print("\n--- View All Profiles ---")
        print("1. Breadth-First Search (BFS)")
        print("2. Depth-First Search (DFS)")
        choice = input("Enter your choice: ")

        if choice not in {"1", "2"}:
            print("Invalid choice.")
            return

        graph = self.profile_manager.graph
        start = self.current_user.get_name() if self.current_user else None

        if not start or start not in graph.get_vertices():
            print("Current user is not valid or graph is empty.")
            return

        if choice == "1":
            result = graph.bfs(start)
        else:
            result = graph.dfs(start)

        print("Profiles in selected order:")
        print(" -> ".join(result))

    def add_friend(self):
        """Add a friend to the current user's friend list."""
        if not self.current_user:
            print("No current user selected.")
            return
        print("\n--- Add a Friend ---")
        friend_name = input("Enter the friend's name: ")

        friend_profile = self.profile_manager.get_profile(friend_name)
        if not friend_profile:
            print(f"Profile for {friend_name} does not exist.")
            return

        self.profile_manager.connect_profiles(self.current_user.get_name(), friend_name)
        print(f"{friend_name} added as a friend!")

    def view_friend_list(self):
        """View the current user's friend list."""
        if not self.current_user:
            print("No current user selected.")
            return
        print("\n--- Friend List ---")
        friends = self.current_user.get_friends()
        print(f"{self.current_user.get_name()}'s friends: {[friend.get_name() for friend in friends]}")

    def view_friends_friend_list(self):
        """View the friend list of each of the current user's friends."""
        if not self.current_user:
            print("No current user selected.")
            return
        print("\n--- Friend's Friend List ---")
        self.profile_manager.get_friends_of_friends(self.current_user.get_name())

    def delete_profile(self):
        """Delete a profile."""
        print("\n--- Delete a Profile ---")
        name = input("Enter the name of the profile to delete: ")
        self.profile_manager.remove_profile(name)
        if self.current_user and self.current_user.get_name() == name:
            self.current_user = None
        print(f"Profile for {name} deleted.")

    def switch_user(self):
        """Switch the current user."""
        print("\n--- Switch User ---")
        name = input("Enter the name of the user to switch to: ")
        new_user = self.profile_manager.get_profile(name)
        if not new_user:
            print(f"Profile for {name} does not exist.")
            return
        self.current_user = new_user
        print(f"Current user switched to {name}.")

    def read_profiles_from_csv(self):
        """Load profiles from a CSV file."""
        print("\n--- Load Profiles from CSV ---")
        file_path = input("Enter the CSV file path: ")
        self.profile_manager.read_profiles_from_csv(file_path)
        print("Profiles loaded from CSV.")

    def create_graph(self):
        """Create a graph of the current user's network."""
        if not self.current_user:
            print("No current user selected.")
            return
        print("\n--- Create Network Graph ---")
        self.profile_manager.create_user_graph(self.current_user.get_name(), depth=1)

    def logout(self):
        """End the program."""
        print("\nLogging out. Goodbye!")
        exit()

    def run(self):
        """Main menu loop."""
        while True:
            print("\n--- Social Media App Menu ---")
            print("1. Create a profile")
            print("2. Modify profile")
            print("3. View all profiles")
            print("4. Add a friend")
            print("5. View your friend list")
            print("6. View your friend's friend list")
            print("7. Delete a profile")
            print("8. Add another profile")
            print("9. Switch the current user")
            print("10. Read profiles from CSV")
            print("11. Create graph of current user's network")
            print("12. Logout")

            choice = input("Enter your choice: ")
            if choice == "1":
                self.create_profile()
            elif choice == "2":
                self.modify_profile()
            elif choice == "3":
                self.view_all_profiles()
            elif choice == "4":
                self.add_friend()
            elif choice == "5":
                self.view_friend_list()
            elif choice == "6":
                self.view_friends_friend_list()
            elif choice == "7":
                self.delete_profile()
            elif choice == "8":
                self.create_profile()
            elif choice == "9":
                self.switch_user()
            elif choice == "10":
                self.read_profiles_from_csv()
            elif choice == "11":
                self.create_graph()
            elif choice == "12":
                self.logout()
            else:
                print("Invalid choice. Please try again.")

# Example usage
# app = SocialMediaApp()
# app.run()


MAIN:FE

In [ ]:
class Main:
    def __init__(self):
        """Initialize the Social Media Application."""
        self.app = SocialMediaApp()

    def run(self):
        """Run the application."""
        print("Welcome to the Social Media Network!")
        print("Before proceeding, you need to create the first user profile.")
        self.app.create_profile()  # Ensures a user profile is created before menu
        print("\nNow you're ready to explore the social media app!")
        self.app.run()  # Start the main menu loop

# Initialize and run the application
if __name__ == "__main__":
    main = Main()
    main.run()
